<a href="https://colab.research.google.com/github/tetsuya335/pxt-ble/blob/master/Trinket_M0_CircuitPython_%E3%83%9C%E3%82%BF%E3%83%B3%E3%82%AD%E3%83%BC%E3%83%9C%E3%83%BC%E3%83%89%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import board
import digitalio
import time
import usb_hid
from adafruit_hid.keyboard import Keyboard
from adafruit_hid.keycode import Keycode

# --- ピン定義 ---
# 黄色いボタンが接続されているピン (CコードのPIN_BUTTON_YEL 0に対応)
PIN_BUTTON_YEL = board.D0
# 赤いボタンが接続されているピン (CコードのPIN_BUTTON_RED 2に対応)
PIN_BUTTON_RED = board.D2

# ボタンのチャタリング防止のためのタイムアウト時間 (ミリ秒)
# CコードのTIMEOUT 500 msecに対応
DEBOUNCE_TIMEOUT_MS = 500

# --- デジタル入力のセットアップ ---
# 黄色いボタンのピンをプルアップ抵抗付きの入力として設定
button_yel = digitalio.DigitalInOut(PIN_BUTTON_YEL)
button_yel.direction = digitalio.Direction.INPUT
button_yel.pull = digitalio.Pull.UP # ボタンが押されるとLOWになるようにプルアップ

# 赤いボタンのピンをプルアップ抵抗付きの入力として設定
button_red = digitalio.DigitalInOut(PIN_BUTTON_RED)
button_red.direction = digitalio.Direction.INPUT
button_red.pull = digitalio.Pull.UP # ボタンが押されるとLOWになるようにプルアップ

# --- USBキーボードのセットアップ ---
# USB HIDキーボードオブジェクトを作成
keyboard = Keyboard(usb_hid.devices)

# --- 状態変数 ---
# 各ボタンが現在押されている状態かどうかを追跡
button_yel_pressed_state = False
button_red_pressed_state = False

# 各ボタンの最後に押された時刻を記録 (チャタリング防止用)
button_yel_last_press_time = 0
button_red_last_press_time = 0

# --- メインループ ---
while True:
    current_time_ms = time.monotonic() * 1000 # 現在時刻をミリ秒で取得

    # --- 黄色いボタンの処理 ---
    if button_yel_pressed_state:
        # ボタンが押された状態の場合、タイムアウトが経過したか確認
        if current_time_ms > button_yel_last_press_time + DEBOUNCE_TIMEOUT_MS:
            button_yel_pressed_state = False # 状態をリセット
    else:
        # ボタンが押されていない状態の場合、物理的にボタンが押されたか確認 (LOWは押された状態)
        if not button_yel.value: # button_yel.valueがFalseならボタンが押されている
            button_yel_pressed_state = True # 状態を「押された」に設定
            button_yel_last_press_time = current_time_ms # 押された時刻を記録
            print("黄色いボタンが押されました: Windows-F4")
            # Windows-F4 ショートカットを送信 (Skypeのミュート切り替えなど)
            keyboard.press(Keycode.GUI, Keycode.F4)
            keyboard.release_all() # 全てのキーをリリース

    # --- 赤いボタンの処理 ---
    if button_red_pressed_state:
        # ボタンが押された状態の場合、タイムアウトが経過したか確認
        if current_time_ms > button_red_last_press_time + DEBOUNCE_TIMEOUT_MS:
            button_red_pressed_state = False # 状態をリセット
    else:
        # ボタンが押されていない状態の場合、物理的にボタンが押されたか確認 (LOWは押された状態)
        if not button_red.value: # button_red.valueがFalseならボタンが押されている
            button_red_pressed_state = True # 状態を「押された」に設定
            button_red_last_press_time = current_time_ms # 押された時刻を記録
            print("赤いボタンが押されました: Windows-L")
            # Windows-L ショートカットを送信 (画面ロック)
            keyboard.press(Keycode.GUI, Keycode.L)
            keyboard.release_all() # 全てのキーをリリース

    # CPU使用率を抑えるために短い遅延を入れる
    time.sleep(0.01)